# Домашнее задание № 5.

Задание посвящено MongoDB. 

## 1. Вариант с pymongo

В рамках домашней работы необходимо:

- подключиться к Mongo из командной строки Linux и загрузить в Mongo текстовый JSON-файл;

In [1]:
!/usr/bin/mongoimport --db movies --collection tags --file $NETOLOGY_DATA/raw_data/simple_tags.json --drop

2019-02-09T17:00:12.169+0300	connected to: localhost
2019-02-09T17:00:12.169+0300	dropping: movies.tags
2019-02-09T17:00:12.997+0300	imported 158680 documents


- выполнить запросы к Mongo через консоль:

In [2]:
import pymongo

In [3]:
from pymongo import MongoClient
db = MongoClient().movies

In [4]:
coll = db.tags

- подсчитайте число элементов в созданной коллекции tags в bd movies

In [5]:
print("tags count: ", coll.find({}).count())

tags count:  158680


/home/truename/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


- подсчитайте число фильмов с конкретным тегом - `woman`

In [6]:
print("woman tags count: ",coll.find({"name": "woman"}).count())

woman tags count:  19


/home/truename/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


- используя группировку данных ($groupby) вывести top-3 самых распространённых тегов

In [7]:
from bson.son import SON

In [8]:
pipeline = [
    {"$group": {"_id": "$name", "tag_count": {"$sum": 1}}},
    {"$sort": SON([("tag_count", -1)])},
    {"$limit": 3}
]

In [9]:
import pprint
pprint.pprint(list(db.tags.aggregate(pipeline)))

[{'_id': 'woman director', 'tag_count': 3115},
 {'_id': 'independent film', 'tag_count': 1930},
 {'_id': 'murder', 'tag_count': 1308}]


## 2. Вариант с командами для mongo

In [10]:
!cat ../shell/hw5.sh

#!/usr/bin/env bash

# команда для загрузки файла в MONGO
/usr/bin/mongoimport --db movies --collection tags --file $NETOLOGY_DATA/raw_data/simple_tags.json --drop

# в файле agg.js три задачи
# - подсчитайте число элементов в созданной коллекции
# - подсчитайте число фильмов с конкретным тегом - `woman`
# - используя группировку данных ($groupby) вывести top-3 самых распространённых тегов
/usr/bin/mongo movies ./agg.js

In [11]:
!cat ../shell/agg.js

/*
        Написать запрос, который выводит общее число тегов
*/
print("tags count: ", db.tags.find({}).count());
/*
        Добавляем фильтрацию: считаем только количество тегов woman
*/
print("woman tags count: ", db.tags.find({'name': 'woman'}).count());
/*
        Очень сложный запрос: используем группировку данных посчитать количество вхождений для каждого тега
        и напечатать top-3 самых популярных
*/

printjson(
        db.tags.aggregate([
                {"$group": {
                                _id: "$name",
                                tag_count: { $sum: 1 }
                           }
                },
                {$sort:{tag_count: -1}},
                {$limit: 3}
        ])['_batch']
);